In [10]:
import time

import torch
import torch.nn as nn

In [2]:
# teacher "models/tf_efficientnet_b4"
results_teacher = [{"epoch": 1,
                    "val_RMAP": 0.277 
                   },
                   {"epoch": 2,
                    "val_RMAP": 0.300846 
                   },
                   {"epoch": 3,
                    "val_RMAP": 0.454 
                   },
                    {"epoch": 4,
                    "val_RMAP": 0.364596 
                   },
                    {"epoch": 5,
                    "val_RMAP": 0.473289
                   }]

In [3]:
# student "models/tf_efficientnet_b0"
# a = 0.5, T = 2.0 
results_student  = [{"epoch": 1,
                    "val_RMAP": 0.288
                   },
                   {"epoch": 2,
                    "val_RMAP": 0.335268 
                   },
                   {"epoch": 3,
                    "val_RMAP": 0.383982 
                   },
                    {"epoch": 4,
                    "val_RMAP": 0.374916 
                   },
                    {"epoch": 5,
                    "val_RMAP": 0.41256
                   }]

In [4]:
model_teacher = torch.load("../experiment/23_October_2023_14_17/model_tf_efficientnet_b4_ep_4.pt")
model_student = torch.load("../experiment/14_September_2023_16_29/model_tf_efficientnet_b0_ep_16.pt")
model_teacher.eval(), model_student.eval()
print('Ok!')

(EfficientNet(
   (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
   (bn1): BatchNormAct2d(
     48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
     (drop): Identity()
     (act): SiLU(inplace=True)
   )
   (blocks): Sequential(
     (0): Sequential(
       (0): DepthwiseSeparableConv(
         (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
         (bn1): BatchNormAct2d(
           48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
           (drop): Identity()
           (act): SiLU(inplace=True)
         )
         (se): SqueezeExcite(
           (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
           (act1): SiLU(inplace=True)
           (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
           (gate): Sigmoid()
         )
         (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn2): BatchNor

In [23]:
def info(model, type_m):
    print(f'Model: {type_m}')
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Общее количество параметров в модели: {total_params}")

    model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())

    model_size_mb = model_size_bytes / (1024 * 1024)
    
    print(f"Общий вес модели: {model_size_mb:.2f} МБ")

    input_data = torch.rand(1, 3, 624, 224).to('cuda')  # Передаем данные на GPU
    model.to('cuda')  # Переводим модель на GPU
    start_time = time.time()
    with torch.no_grad():
        output = model(input_data)
    end_time = time.time()
    inference_time = end_time - start_time
    print(f"Время предсказания: {inference_time:.4f} секунд на GPU")

    model.to('cpu')  # Вернем модель на CPU
    input_data = input_data.to('cpu')  # Вернем данные на CPU
    start_time = time.time()
    with torch.no_grad():
        output = model(input_data)
    end_time = time.time()
    inference_time = end_time - start_time
    print(f"Время предсказания: {inference_time:.4f} секунд на CPU")
    
    print("-----------------------------------------")


In [24]:
info(model_teacher, 'teacher')
info(model_student, 'student')

Model: teacher
Общее количество параметров в модели: 18021968
Общий вес модели: 68.75 МБ
Время предсказания: 0.0201 секунд на GPU
Время предсказания: 0.2720 секунд на CPU
-----------------------------------------
Model: student
Общее количество параметров в модели: 4345732
Общий вес модели: 16.58 МБ
Время предсказания: 0.0090 секунд на GPU
Время предсказания: 0.1121 секунд на CPU
-----------------------------------------
